Sealevel monitor
========

This document describes the estimate of power to detect a trend break in sea-surface heights along the Dutch coast.

In [1]:
# this is a list of packages that are used in this notebook
# these come with python
import io
import zipfile
import functools
import bisect
import datetime
import pathlib


# you can install these packages using pip or anaconda
# (requests numpy pandas bokeh pyproj statsmodels)

# for downloading
import requests
import netCDF4

# computation libraries
import numpy as np
import pandas as pd


# coordinate systems
import pyproj 

# statistics
import statsmodels.api as sm
import statsmodels.multivariate.pca
import statsmodels.tsa.seasonal


# plotting
import bokeh.io
import bokeh.plotting
import bokeh.tile_providers
import bokeh.palettes

import windrose
import matplotlib.colors
import matplotlib.cm
import matplotlib.pyplot as plt
matplotlib.projections.register_projection(windrose.WindroseAxes)
import cmocean.cm

# displaying things
from ipywidgets import Image
import IPython.display

%load_ext rpy2.ipython

In [2]:
# Some coordinate systems
WEBMERCATOR = pyproj.Proj(init='epsg:3857')
WGS84 = pyproj.Proj(init='epsg:4326')

# If this notebook is not showing up with figures, you can use the following url:
# https://nbviewer.ipython.org/github/openearth/notebooks/blob/master/sealevelmonitor.ipynb
bokeh.io.output_notebook()
# we're using matplotlib for polar plots (non-interactive)
%matplotlib inline
# does not work properly
# %matplotlib notebook


Loading BokehJS ...

In [3]:
mean_df = pd.read_csv('../../dutch-sea-level-monitor-export-2019-01-28.csv', comment='#')
mean_df.head()

,index,year,height,u2,v2,predicted_linear_with_wind,predicted_linear,predicted_linear_mean_wind,predicted_linear_mean_wind_ci_025,predicted_linear_mean_wind_ci_975,predicted_linear_mean_wind_pi_025,predicted_linear_mean_wind_pi_975,date
0,28,1890,-194.666667,3.348202,0.669777,-164.417811,-167.776708,-173.414888,-182.368336,-164.461441,-220.581336,-126.248440,1890-01-01
1,29,1891,-179.000000,3.348202,0.669777,-160.938261,-163.580429,-171.558355,-180.412383,-162.704328,-218.706032,-124.410679,1891-01-01
2,30,1892,-166.500000,3.348202,0.669777,-158.657457,-160.795674,-169.701822,-178.456697,-160.946948,-216.830979,-122.572665,1892-01-01
3,31,1893,-142.166667,3.348202,0.669777,-157.623290,-159.519472,-167.845290,-176.501287,-159.189293,-214.956179,-120.734400,1893-01-01
4,32,1894,-141.833333,3.348202,0.669777,-157.742934,-159.678570,-165.988757,-174.546162,-157.431352,-213.081631,-118.895882,1894-01-01


In [4]:
def timeseries_plot():
    # show all the stations, including the mean
    title = 'Sea-surface height for Dutch tide gauges [{year_min} - {year_max}]'.format(
        year_min=mean_df.year.min(),
        year_max=mean_df.year.max() 
    )
    fig = bokeh.plotting.figure(title=title, x_range=(1860, 2020), plot_width=900, plot_height=400)
    colors = bokeh.palettes.Accent7
    # no yellow
    fig.line(mean_df.year, mean_df.height, line_width=1, alpha=0.7, color='black', legend='Mean')
    fig.legend.location = "bottom_right"
    fig.yaxis.axis_label = 'waterlevel [mm] above NAP'
    fig.xaxis.axis_label = 'year'
    fig.legend.click_policy = "hide"
    return fig


In [5]:
bokeh.io.show(timeseries_plot())

Methods
=====
The [power analysis](https://books.google.nl/books?id=FnW8tAEACAAJ&dq=isbn:0805802835&hl=en&sa=X&ved=0ahUKEwiAgLnu_ZrhAhXJYlAKHbtqDD0Q6AEIKjAA), as known in experimental design, allows to determine the probability of detecting an effect of a given size under the assumption that the effect is present. 

The power is a function of the effect size. The effect size $f^2$ is defined as the $\frac{R^2_{AB}-R^2_{A}}{1-R^2_{AB}}$. Where $R^2_{A}$ is the $R^2$ for the  model without acceleration and $R^2_{AB}$ is the $R^2$ for the model with acceleration.

We use the same model as in the main document. To give an idea on how much extra sea-level rise we would have needed to increase the power, we run two extra experiments. We create two derived variants of the dataset with more sea-level rise. A variant with 0.2mm/year extra sea level rise that we refer to as `low` and a variant with 0.5mm/year extra sea-level rise after 1993 that we refer to has `high`. 

For a linear model typically a $f^2$ effect size of 0.02 is referred to as small, 0.15 as medium and 0.35 as high. 


In [6]:
# define the statistical model, see dutch-sealevel-monitor for explanation.
def linear_model(df, with_wind=True, with_ar=True, with_nodal=True, acceleration=None):
    y = df['height']
    X = np.c_[
        df['year']-1970
    ]
    month = np.mod(df['year'], 1) * 12.0
    names = ['Constant', 'Trend']
    if acceleration == 'broken':
        # add broken trend model
        X = np.c_[
            X,
            (df['year'] > 1993) * (df['year'] - 1993)
        ]
        names += ['+trend (1993)']
    if acceleration == 'polynomial':
        # add polynomial acceleration
        X = np.c_[
            X,
            (df['year'] - 1970) * (df['year'] - 1970)
        ]
        names += ['Acceleration']
    
    if with_nodal:
        X = np.c_[
            X, 
            np.cos(2*np.pi*(df['year']-1970)/18.613),
            np.sin(2*np.pi*(df['year']-1970)/18.613)
        ]
        names += ['Nodal U', 'Nodal V']
    if with_wind:
        X = np.c_[
            X, 
            df['u2'],
            df['v2']
        ]
        names.extend(['Wind $u^2$', 'Wind $v^2$'])
    X = sm.add_constant(X)
    if with_ar:
        model = sm.GLSAR(y, X, missing='drop', rho=1)
        fit = model.fit(cov_type='HC0')
    else:
        model = sm.OLS(y, X, missing='drop')
        fit = model.fit() 
    return fit, names

In [7]:
# these are all the variants that we compute
fits = [
    {
        "id": "linear-wind-ar",
        "name": "Linear Wind Autoregression",
        "args": dict(with_wind=True, with_ar=True)
    },
    {
        "id": "linear-wind-noar",
        "name": "Linear Wind No Autoregression",
        "args": dict(with_wind=True, with_ar=False)
    },
    {
        "id": "broken-wind-ar",
        "name": "Broken Wind Autoregression",
        "args": dict(with_wind=True, with_ar=True, acceleration='broken')
    },
    {
        "id": "broken-wind-noar",
        "name": "Broken Wind No Autoregression",
        "args": dict(with_wind=True, with_ar=False, acceleration='broken')
    },
    {
        "id": "polynomial-wind-ar",
        "name": "Polynomial Wind Autoregression",
        "args": dict(with_wind=True, with_ar=True, acceleration='polynomial')
    },
    {
        "id": "polynomial-wind-noar",
        "name": "Polynomial Wind No Autoregression",
        "args": dict(with_wind=True, with_ar=False, acceleration='polynomial')
    }
]

# low scenario (extra acceleration of 0.2mm per year)
low_df = mean_df.copy()
low_df['height'] += (low_df['year'] > 1993) * (low_df['year'] - 1993) * 0.2
# high scenario (extra acceleration of 0.5mm per year)
high_df = mean_df.copy()
high_df['height'] += (high_df['year'] > 1993) * (high_df['year'] - 1993) * 0.5

scenarios = [
    {
        "id": "mean",
        "df": mean_df
    },
    {
        "id": "low",
        "df": low_df
    }, 
    {
        "id": "high",
        "df": high_df
    }
]

result = []
for fit_row in fits:
    for scenario_row in scenarios:
        fit, fit_names = linear_model(scenario_row['df'], **fit_row['args'])
        result.append({
            "fit": fit_row['id'],
            "scenario": scenario_row['id'],
            "r2": fit.rsquared,
            "df_model": fit.df_model,
            "df_resid": fit.df_resid
        })
results = pd.DataFrame(result).set_index(['fit', 'scenario'])
results

df_model  df_resid        r2
fit                  scenario                              
linear-wind-ar       mean           5.0     121.0  0.907247
                     low            5.0     121.0  0.908137
                     high           5.0     121.0  0.909069
linear-wind-noar     mean           5.0     122.0  0.908488
                     low            5.0     122.0  0.909408
                     high           5.0     122.0  0.910389
broken-wind-ar       mean           6.0     120.0  0.908396
                     low            6.0     120.0  0.910097
                     high           6.0     120.0  0.912606
broken-wind-noar     mean           6.0     121.0  0.909417
                     low            6.0     121.0  0.911072
                     high           6.0     121.0  0.913514
polynomial-wind-ar   mean           6.0     120.0  0.907288
                     low            6.0     120.0  0.908139
                     high           6.0     120.0  0.909239
polynomial-wind-noar mean           6.0     121.0  0.908668
                     low            6.0     121.0  0.909439
                     high           6.0     121.0  0.910425

In [8]:
# Now we can compute the expected power for all the different models
# load the power library in R
%R library(pwr)

a = results.loc[('linear-wind-ar', 'mean')]

comparisons = [
    ('broken-wind-ar', 'mean'),
    ('broken-wind-ar', 'low'),
    ('broken-wind-ar', 'high'),
    ('polynomial-wind-ar', 'mean'),
    ('polynomial-wind-ar', 'low'),
    ('polynomial-wind-ar', 'high')
]

r2dict = lambda x: { key : x.rx2(key)[0] for key in x.names }
powers = []
for comparison in comparisons:
    ab = results.loc[comparison]
    f2 = (ab.r2 - a.r2)/(1-ab.r2)
    u = ab.df_model - a.df_model
    v = a.df_resid
    # compute 
    result = %R -i f2 -i u -i v pwr.f2.test(u, v, f2, 0.05)
    result = r2dict(result)
    result['comparison'] = comparison
    result.pop('method')
    result['$R^2_{AB}$'] = ab.r2
    result['$R^2_{A}$'] = a.r2
    powers.append(result)
pd.DataFrame(powers).set_index('comparison')

,$R^2_{AB}$,$R^2_{A}$,f2,power,sig.level,u,v
comparison,,,,,,,
"(broken-wind-ar, mean)",0.908396,0.907247,0.012543,0.234096,0.05,1.0,121.0
"(broken-wind-ar, low)",0.910097,0.907247,0.031708,0.499731,0.05,1.0,121.0
"(broken-wind-ar, high)",0.912606,0.907247,0.061330,0.777787,0.05,1.0,121.0
"(polynomial-wind-ar, mean)",0.907288,0.907247,0.000448,0.056230,0.05,1.0,121.0
"(polynomial-wind-ar, low)",0.908139,0.907247,0.009713,0.191796,0.05,1.0,121.0
"(polynomial-wind-ar, high)",0.909239,0.907247,0.021949,0.370899,0.05,1.0,121.0


# Conclusion
One can see in the table above that the broken linear model is more powerful in detecting a recent acceleration than the polynomial model. 

The current broken linear acceleration is considered 'small'. The current polynomial acceleration is almost non-existent (very small deceleration).

If there would have been an extra sea-level rise of 0.2 mm/yr over the period 1993-2017 then the probability of detecting the sea-level rise would increase to 50%. If the sea-level rise would have been 0.5mm/year higher we would have detected the sea-level rise with a probability of 78%. 

Because there is almost no polynomial acceleration over the full time window the power is only 5.6%. With an extra 0.5mm/year sea-level rise the power goes up to 37% with the polynomial model.